In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 16, 12
import glob
from skimage import data, io, segmentation, color
from skimage.future import graph

<p><b><i>Ucitavanje slika<i/><b><p>

In [2]:
def load_image(path):
    img = cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

<p><b><i>Zdrave vocke<i/><b/><p/>

In [9]:
freshImg = [img for img in glob.glob("images/dataset/train/fresh/*.jpg")]
freshImg.sort()
fresh = []
for img in freshImg:
    temp = load_image(img)
    fresh.append(temp)

<p><b><i>Trule vocke<i/><b/><p/>

In [10]:
rottenImg = [img for img in glob.glob("images/dataset/train/rotten/*.jpg")]
rottenImg.sort()
rotten = []
for img in rottenImg:
    temp = load_image(img)
    rotten.append(temp)

<p><b><i>Podela slika na regione preko k-means<i/><b/><p/>

In [12]:
freshReg = []
for imgFresh in fresh:
    img = imgFresh.copy()
    Z = img.reshape((-1, 3))
    Z = np.float32(Z)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 6
    ret, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    ## Vracanje originalne slike
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))

    freshReg.append(res2)

In [13]:
rottenReg = []
for imgRotten in fresh:
    img = imgRotten.copy()
    Z = img.reshape((-1, 3))
    Z = np.float32(Z)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 6
    ret, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    ## Vracanje originalne slike
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))

    rottenReg.append(res2)

<p><b><i>Spajanje slicnih regiona preko merge algoritma RAG(Region Adjacent Graphs)<i/><b/><p/>

In [28]:
## poziva se nakon spajanja dva regiona, gde se racuna weight regiona koji je sada susedan
## novonastalom regionu
def _weight_mean_color(graph, src, dst, n):
    diff = graph.node[dst]['mean color'] - graph.node[n]['mean color']
    diff = np.linalg.norm(diff)
    return {'weight': diff}

## poziva se uoci spajanja dva regiona
def merge_mean_color(graph, src, dst):
    graph.node[dst]['total color'] += graph.node[src]['total color']
    graph.node[dst]['pixel count'] += graph.node[src]['pixel count']
    graph.node[dst]['mean color'] = (graph.node[dst]['total color'] /
                                     graph.node[dst]['pixel count'])

In [26]:
freshMerge = []
for img in fresh:
    ## segmentira u koloru
    labels = segmentation.slic(img, compactness=30, n_segments=40)
    ## od slike pravi korespondirajuci RAG graf
    g = graph.rag_mean_color(img, labels)
    ## vrsi hijerarhijsko spajanje grafa
    labels2 = graph.merge_hierarchical(labels, g, thresh=75, rag_copy=False,
                                       in_place_merge=True,
                                       merge_func=merge_mean_color,
                                       weight_func=_weight_mean_color)
    out = color.label2rgb(labels2, img, kind='avg')
    out = segmentation.mark_boundaries(out, labels2, (0, 0, 0))
    freshMerge.append(out)

In [ ]:
rottenMerge = []
for img in rottenReg:
    labels = segmentation.slic(img, compactness=30, n_segments=400)
    g = graph.rag_mean_color(img, labels)
    labels2 = graph.merge_hierarchical(labels, g, thresh=75, rag_copy=False,
                                       in_place_merge=True,
                                       merge_func=merge_mean_color,
                                       weight_func=_weight_mean_color)
    out = color.label2rgb(labels2, img, kind='avg')
    out = segmentation.mark_boundaries(out, labels2, (0, 0, 0))
    rottenMerge.append(out)